In [1]:
#importing libraries
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [3]:
#importing the dataset
data = pd.read_csv("kc_house_data.csv")

In [19]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [15]:
column_indices = {col: idx for idx, col in enumerate(data.columns)}
column_indices #personal use to easy index

{'id': 0,
 'date': 1,
 'price': 2,
 'bedrooms': 3,
 'bathrooms': 4,
 'sqft_living': 5,
 'sqft_lot': 6,
 'floors': 7,
 'waterfront': 8,
 'view': 9,
 'condition': 10,
 'grade': 11,
 'sqft_above': 12,
 'sqft_basement': 13,
 'yr_built': 14,
 'yr_renovated': 15,
 'zipcode': 16,
 'lat': 17,
 'long': 18,
 'sqft_living15': 19,
 'sqft_lot15': 20}

In [4]:
#extracting the useful columns
X = data.iloc[:,3:].values
X = X[:,np.r_[0:13,14:18]]
y = data.iloc[:,2].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

In [6]:
# Scaling the features
xscaler = MinMaxScaler(feature_range=(0,1))
X_train = xscaler.fit_transform(X_train)
X_test = xscaler.transform(X_test)

# Scaling the target
yscaler = MinMaxScaler(feature_range=(0,1))
y_train = yscaler.fit_transform(y_train.reshape(-1,1))
y_test = yscaler.transform(y_test.reshape(-1,1))

In [7]:
# building the artificial neural network
model = Sequential()
model.add(Dense(units = 64, kernel_initializer="uniform",activation = "relu",input_dim = 17))
model.add(Dense(units = 16, kernel_initializer="uniform",activation = "relu"))
model.add(Dense(units= 1,kernel_initializer = "uniform",activation ="relu"))
model.compile(optimizer=Adam(lr=0.001),loss = "mse",metrics=["mean_absolute_error"])

In [8]:
# Training the Artificial Neural Network
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test,y_test))

Epoch 1/100
541/541 [==============================] - 3s 4ms/step - loss: 0.0011 - mean_absolute_error: 0.0198 - val_loss: 5.7284e-04 - val_mean_absolute_error: 0.0145
Epoch 2/100
541/541 [==============================] - 2s 3ms/step - loss: 6.2418e-04 - mean_absolute_error: 0.0153 - val_loss: 5.0714e-04 - val_mean_absolute_error: 0.0138
Epoch 3/100
541/541 [==============================] - 2s 3ms/step - loss: 5.5579e-04 - mean_absolute_error: 0.0143 - val_loss: 4.6047e-04 - val_mean_absolute_error: 0.0131
Epoch 4/100
541/541 [==============================] - 1s 3ms/step - loss: 5.1142e-04 - mean_absolute_error: 0.0136 - val_loss: 4.6164e-04 - val_mean_absolute_error: 0.0122
Epoch 5/100
541/541 [==============================] - 1s 2ms/step - loss: 4.8226e-04 - mean_absolute_error: 0.0133 - val_loss: 3.9530e-04 - val_mean_absolute_error: 0.0116
Epoch 6/100
541/541 [==============================] - 1s 2ms/step - loss: 4.5479e-04 - mean_absolute_error: 0.0129 - val_loss: 3.9146e-04 

In [9]:
# Making predictions on the test set while reversing the scaling
y_test = yscaler.inverse_transform(y_test)
prediction = yscaler.inverse_transform(model.predict(X_test))

#Computing the error rate
error = abs(prediction - y_test)/y_test
print(np.mean(error))

136/136 [==============================] - 0s 1ms/step
0.23080676685840726
